In [36]:
!pip install -q -U kaggle_environments

In [37]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0


In [38]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем
агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [39]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, 3)

Overwriting copy_opponent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [40]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[1, None]]

In [41]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[-99.0, 99.0]]

Добавляем разных агентов

In [42]:
import random

def rock_agent(observation, configuration):
    """
    Агент, который всегда выбирает камень.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: 0 (камень)
    """
    return 0

def paper_agent(observation, configuration):
    """
    Агент, который всегда выбирает бумагу.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: 1 (бумага)
    """
    return 1

def scissors_agent(observation, configuration):
    """
    Агент, который всегда выбирает ножницы.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: 2 (ножницы)
    """
    return 2

def random_agent(observation, configuration):
    """
    Агент, который выбирает случайный ход (камень, бумага или ножницы).

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: Случайный ход (0, 1 или 2)
    """
    return random.choice([0, 1, 2])

def copy_opponent_agent(observation, configuration):
    """
    Агент, который копирует последний ход противника.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: Последний ход противника или случайный ход, если история пуста.
    """
    if observation.get('step', 0) > 0:
        return observation.get('lastOpponentAction', random.choice([0, 1, 2]))
    else:
        return random.choice([0, 1, 2])

def beat_last_agent(observation, configuration):
    """
    Агент, который выбирает ход, который бьет последний ход противника.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: Ход, который бьет последний ход противника, или случайный ход, если история пуста.
    """
    if observation.get('step', 0) > 0:
        return (observation.get('lastOpponentAction', 0) + 1) % 3
    else:
        return random.choice([0, 1, 2])

def cycle_agent(observation, configuration):
    """
    Агент, который циклически выбирает ходы (камень, бумага, ножницы).

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: Следующий ход в цикле (камень, бумага, ножницы).
    """
    if observation.get('step', 0) == 0:
        return 0
    else:
        return (observation.get('lastOpponentAction', 0) + 1) % 3

def counter_cycle_agent(observation, configuration):
    """
    Агент, который выбирает ход, который бьет циклический ход противника.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: Ход, который бьет следующий ход в цикле противника.
    """
    if observation.get('step', 0) == 0:
        return 0
    else:
        return (observation.get('lastOpponentAction', 0) + 2) % 3

def frequency_agent(observation, configuration):
    """
    Агент, который выбирает наиболее часто используемый противником ход.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: Наиболее часто используемый противником ход или случайный ход, если история пуста.
    """
    if observation.get('step', 0) > 0:
        counts = [0, 0, 0]
        for action in observation.get('history', []):
            counts[action] += 1
        return counts.index(max(counts))
    else:
        return random.choice([0, 1, 2])

def beat_frequency_agent(observation, configuration):
    """
    Агент, который выбирает ход, который бьет наиболее часто используемый противником ход.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: Ход, который бьет наиболее часто используемый противником ход, или случайный ход, если история пуста.
    """
    if observation.get('step', 0) > 0:
        counts = [0, 0, 0]
        for action in observation.get('history', []):
            counts[action] += 1
        return (counts.index(max(counts)) + 1) % 3
    else:
        return random.choice([0, 1, 2])

def mixed_strategy_agent_rock(observation, configuration):
    """
    Агент, который использует смешанную стратегию для выбора хода.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: Ход с определенным смещением.
    """
    # Смешанная стратегия с преимуществом для камня (0)
    mixed_choices = [0, 0, 0, 1, 1, 2]
    return random.choice(mixed_choices)

def mixed_strategy_agent_paper(observation, configuration):
    """
    Агент, который использует смешанную стратегию для выбора хода.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: Ход с определенным смещением.
    """
    # Смешанная стратегия с преимуществом для камня (0)
    mixed_choices = [0, 0, 1, 1, 1, 2]
    return random.choice(mixed_choices)

def mixed_strategy_agent_scissors(observation, configuration):
    """
    Агент, который использует смешанную стратегию для выбора хода.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: Ход с определенным смещением.
    """
    # Смешанная стратегия с преимуществом для камня (0)
    mixed_choices = [0, 0, 1, 1, 2, 2]
    return random.choice(mixed_choices)

def adaptive_agent(observation, configuration):
    """
    Адаптивный агент, который пытается предсказать следующий ход противника.

    :param observation: Информация о текущем состоянии игры.
    :param configuration: Конфигурация игры.
    :return: Ход, который бьет предсказанный ход противника, или случайный ход, если история недостаточно длинная.
    """
    if observation.get('step', 0) > 1:
        if observation.get('lastOpponentAction', 0) == observation.get('history', [0, 0])[-2]:
            return (observation.get('lastOpponentAction', 0) + 1) % 3
        else:
            return 0
    else:
        return 0


In [43]:
# Список агентов
agents = [
    rock_agent,
    paper_agent,
    scissors_agent,
    random_agent,
    copy_opponent_agent,
    beat_last_agent,
    cycle_agent,
    counter_cycle_agent,
    frequency_agent,
    beat_frequency_agent,
    mixed_strategy_agent_paper,
    mixed_strategy_agent_rock,
    mixed_strategy_agent_scissors,
    adaptive_agent
]

def check_agents(agents):
    """
    Проверяет, что все агенты возвращают допустимые ходы (0, 1 или 2).

    :param agents: Список агентов для проверки.
    """
    errors_found = False
    for agent in agents:
        try:
            result = agent({'step': 0, 'lastOpponentAction': None, 'history': []}, {})
            if result not in [0, 1, 2]:
                print(f"Agent {agent.__name__} returned invalid move: {result}")
                errors_found = True
        except Exception as e:
            print(f"Agent {agent.__name__} raised an exception: {e}")
            errors_found = True

    # Если ошибок не обнаружено, выводим сообщение
    if not errors_found:
        print("Ошибок не обнаружено")

# Проверка всех агентов
check_agents(agents)

Ошибок не обнаружено


In [44]:
from kaggle_environments import make, evaluate
from collections import defaultdict

# Запуск турнира
results = []
for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        agent1 = agents[i]
        agent2 = agents[j]
        match_result = evaluate(
            "rps",  # environment to use
            [agent1, agent2],  # agents to evaluate
            configuration={"episodeSteps": 100}  # number of episodes
        )
        results.append((agent1.__name__, agent2.__name__, match_result))

# Вывод результатов
for result in results:
    print(f"Agent 1: {result[0]}, Agent 2: {result[1]}, Result: {result[2]}")

# Подсчет результатов
agent_scores = defaultdict(lambda: {"wins": 0, "losses": 0, "draws": 0})

for result in results:
    agent1_name = result[0]
    agent2_name = result[1]
    match_result = result[2][0]

    # Проверка на корректность результатов
    if match_result[0] is None or match_result[1] is None:
        print(f"Error in match between {agent1_name} and {agent2_name}: One of the agents returned None.")
        continue

    if match_result[0] > match_result[1]:
        agent_scores[agent1_name]["wins"] += 1
        agent_scores[agent2_name]["losses"] += 1
    elif match_result[0] < match_result[1]:
        agent_scores[agent1_name]["losses"] += 1
        agent_scores[agent2_name]["wins"] += 1
    else:
        agent_scores[agent1_name]["draws"] += 1
        agent_scores[agent2_name]["draws"] += 1

# Вывод результатов
for agent, scores in agent_scores.items():
    print(f"Agent: {agent}, Wins: {scores['wins']}, Losses: {scores['losses']}, Draws: {scores['draws']}")

Agent 1: rock_agent, Agent 2: paper_agent, Result: [[-99.0, 99.0]]
Agent 1: rock_agent, Agent 2: scissors_agent, Result: [[99.0, -99.0]]
Agent 1: rock_agent, Agent 2: random_agent, Result: [[0, 0]]
Agent 1: rock_agent, Agent 2: copy_opponent_agent, Result: [[0, 0]]
Agent 1: rock_agent, Agent 2: beat_last_agent, Result: [[-99.0, 99.0]]
Agent 1: rock_agent, Agent 2: cycle_agent, Result: [[-98.0, 98.0]]
Agent 1: rock_agent, Agent 2: counter_cycle_agent, Result: [[98.0, -98.0]]
Agent 1: rock_agent, Agent 2: frequency_agent, Result: [[0, 0]]
Agent 1: rock_agent, Agent 2: beat_frequency_agent, Result: [[-97.0, 97.0]]
Agent 1: rock_agent, Agent 2: mixed_strategy_agent_paper, Result: [[-44.0, 44.0]]
Agent 1: rock_agent, Agent 2: mixed_strategy_agent_rock, Result: [[0, 0]]
Agent 1: rock_agent, Agent 2: mixed_strategy_agent_scissors, Result: [[0, 0]]
Agent 1: rock_agent, Agent 2: adaptive_agent, Result: [[-97.0, 97.0]]
Agent 1: paper_agent, Agent 2: scissors_agent, Result: [[-99.0, 99.0]]
Agent 